In [1]:
import wordcloud
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from matplotlib import rcParams, pyplot as plt
import csv #调用数据保存文件
import pandas as pd #用于数据输出

#设置字体
rcParams["font.size"] = 8
rcParams['figure.figsize'] = (10.0, 10.0) 
plt.style.use('ggplot')
plt.figure(figsize=(15,15),dpi=500) #设置图片大小和像素

#下载所需数据集
#nltk.download()

#架在停用词
stop_words = list(stopwords.words('german')) # 停用词表，去掉不需要的词
for i in range(len(stop_words)):
    stop_words.append(stop_words[i].capitalize())
stop_words.extend(('immer','Chinas','müssen','wurde','schon','heute','viele','Jahren','gerade','Prozent','wäre','dafür','eigenen','weniger','wurden','Milliarden','sehen','kommen','großen','gegenüber','lange','große','brauchen','steht','inzwischen','lässt','Jahre','unserer','vielen','Blick','macht','wirklich','Beispiel','bereits','darauf','wohl','besonders','ersten','fast','geben','tun','hätte','wenig','denen','vergangenen','sollten','damals','darüber','eigene','sogar','klar','lassen','seien','heißt','stellen','kommt','sollen','natürlich','erste','Wochen','Fall','davon','einfach','gehen','stehen','Wer','innen','Woche','gemacht','bleiben','pro','derzeit','täglich','halten','darf','später','deutlich','dabei','braucht','scheint','zurück','daher','sagte','allerdings','helfen','sieht','nie','U','Vorjahr','Jahren','wurde','immer','Prozent','wurden','müssen','Jahre','viele','sagte','drei','gegenüber','heute','Millionen','großen','fünf','schon','Milliarden','bereits','seien','steht','Dienstag','unserer','dafür','Uhr','gerade','weiterhin','insbesondere','wäre','sieht','Mio','kommt','insgesamt','Ve','sogar','große','deutlich ','sagen','kommen','dabei','vielen','vier','aufgrund','bzw','sollen','lange','worden','davon','konnte','Donald','lassen','bleiben','bleibt','derzeit','geworden','ersten','allerdings','sehen','größten','fast','Tag','sollten','wenig','darauf','Angela','heißt','zehn','gilt','letzten','wegen','stehen','jedoch','Dezember','denen','İş','Bankası','Beispiel','liegt','Seite','Woche','macht','eigenen','tun','stellen','nie','a','sowie','jahr','Jahr','t€','T€','gfr','mehr','prozent','uhr','sei','ab','AG','T','gut','mal','seit','sagt','gibt','ganz','wer','ende','teur','rund','jahr','ja','dach','DACH','beim','jahr','geht','gar','zwei','erst','etwa'))

all_words=[]
emo_dict={}

#文本清理
def cleanText(text):
    #分句
    sents=sent_tokenize(text)
    for sent in sents:
        #分词、去除停用词
        word_tokens=word_tokenize(sent)
        for w in word_tokens:
            if (w not in stop_words) and w[0].isalpha():
                all_words.append(w)
    #print(all_words[0])

#统计词频并绘制图表
def printFreq():
    freq_dist = FreqDist(all_words)
    i=0
    '''
    labels=[]
    x=[]
    y=[]
    for k in freq_dist:
        i+=1
        if i>50:
            break
        labels.append(k+' ('+str(i)+')')
        x.append(i)
        y.append(freq_dist[k])
    plt.plot(x,y)
    plt.xticks(range(50),labels=labels,rotation=90) #x轴坐标轴
    '''
    for k in freq_dist:
        i+=1
        if i>50:
            break
        print("'"+k+"',",end='')

#绘制词云图
def drawCloud(beforeText,afterText):
    wc = wordcloud.WordCloud(font_path="msyh.ttc",
                            width = 1000,
                            height = 700,
                            background_color='white',
                            max_words=50,
                            stopwords=stop_words)
    wc.generate(beforeText) # 加载词云文本
    wc.to_file("cloud_before.png") # 保存词云文件
    wc.generate(afterText)
    wc.to_file("cloud_after.png")

#构造情感词典
def init_dict():
    emo=0
    count=0

    #打开两个字典文件
    for file in ['SentiWS_v2.0_Negative.txt','SentiWS_v2.0_Positive.txt']:
        f=open(file)
        lines=f.readlines()
        for line in lines:
            words=line.split('	')
            word_cha=words[0]
            weight=words[1]
            other_forms=words[2]

            if '-' in weight:
                w = -1
            else:
                w = 1
            emo+=float(weight)
            count+=1 #计算总次数

            index=word_cha.find('|')
            if 0 and '选择计算情感的方法--------------------------------------------------':
                emo_dict[word_cha[:index]]=w
                words=other_forms.split(',')
                for word in words:
                    word = word.strip()
                    emo_dict[word]=w
            else:
                emo_dict[word_cha[:index]]=weight
                words=other_forms.split(',')
                for word in words:
                    word = word.strip()
                    emo_dict[word]=weight
        f.close()
    #print(emo_dict)
    print('词典中平均每词带有情感：'+str(emo/count)[:9]+'\n')
#计算文本情感
def count_emo(s):
    posi_emo=0
    nega_emo=0
    posi_count=0
    nega_count=0

    emo=0
    count=0
    '''
    for word in all_words:
        #当字典中有该词时，改变emo值
        if word in emo_dict:
            count=count+1
            emo+=float(emo_dict[word])
    '''
    for word in all_words:
        #当字典中有该词时，改变emo值
        if word in emo_dict:
            amount=float(emo_dict[word])
            if amount>0:
                posi_count+=1
                posi_emo+=amount
            else:
                nega_count+=1
                nega_emo+=amount
            
    print(s+'，积极情感词数：'+str(posi_count))
    print(s+'，积极情感总值：'+str(posi_emo)[:9])
    print(s+'，积极词平均情感：'+str(posi_emo/posi_count)[:9]+'\n')
    print(s+'，消极情感词数：'+str(nega_count))
    print(s+'，消极情感总值：'+str(nega_emo)[:9])
    print(s+'，消极词平均情感：'+str(nega_emo/nega_count)[:9]+'\n')

def main():
    # 分别从语料库读取疫情前后的文本
    with open("myCorpus_before.txt",encoding="utf-8") as f:
        beforeText = f.read()
    with open("myCorpus_after.txt",encoding="utf-8") as f:
        afterText = f.read()

    #init_dict()

    cleanText(beforeText)
    printFreq()
    #count_emo('疫情前')

    all_words.clear()

    cleanText(afterText)
    printFreq()
    #count_emo('疫情后')

    #drawCloud(beforeText,afterText)

main()

'China','Deutschland','Europa','USA','TEUR','Unternehmen','Welt','İşbank','Geschäftsjahr','EU','Euro','neue','Höhe','deutschen','neuen','Entwicklung','Merkel','Trump','Land','Peking','EUR','deutsche','Wirtschaft','Schicksalsgemeinschaft','Xi','chinesische','Ende','Berlin','Bank','Menschen','Türkei','Länder','chinesischen','Zeit','Präsident','Risiken','Verbindlichkeiten','Zukunft','Bereich','Wohnungen','Investitionen','Regierung','Staaten','Russland','Politik','Wachstum','Zusammenarbeit','zusammen','Vergleich','Li','China','Deutschland','Europa','USA','Welt','Virus','Russland','EU','Menschen','Land','Pandemie','chinesischen','deutschen','deutsche','Berlin','Politik','Wirtschaft','neue','Länder','Frage','Schicksalsgemeinschaft','Unternehmen','Ukraine','Italien','BSZ','neuen','Deutschlands','Taiwan','Ende','Corona','Demokratie','Weber','Zeit','Peking','Deutschen','globalen','Xi','Menschheit','Krise','Europas','Ländern','Zukunft','Staaten','politischen','chinesische','Entwicklung','Westen'

<Figure size 7500x7500 with 0 Axes>